In [1]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=f914239b9bf6fc1dce2b227f03829d5952c0d89406035ba17e76f258c91be45b
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install language_tool_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
# !pip install librosa nltk language-tool-python fasttext
import os
import numpy as np
import librosa
import nltk
import re
import fasttext
import fasttext.util
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from langdetect import detect
import language_tool_python
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Concatenate, Flatten

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os

2025-05-11 13:49:04.569979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746971344.723393      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746971344.767513      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Lowercase, remove numbers/punctuation, grammar correct
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)

    # Tokenize, remove stopwords, lemmatize
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(words)


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
import os
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import matplotlib.pyplot as plt

def get_patient_timestamps(transcript_path):
    df = pd.read_csv(transcript_path, sep='\t')
    df.columns = df.columns.str.strip().str.lower()
    df['speaker'] = df['speaker'].str.strip().str.lower()
    patient_segments = df[df['speaker'] == 'participant'][['start_time', 'stop_time']]
    return list(patient_segments.itertuples(index=False, name=None))

def load_patient_audio(audio_path, timestamps):
    audio, sr = librosa.load(audio_path, sr=None)
    patient_audio = []
    for start, stop in timestamps:
        start_sample = int(start * sr)
        stop_sample = int(stop * sr)
        if start_sample < stop_sample:  # avoid invalid segments
            patient_audio.append(audio[start_sample:stop_sample])
    if not patient_audio:
        raise ValueError("No valid participant audio segments found.")
    patient_audio = np.concatenate(patient_audio)
    return patient_audio, sr

def segment_audio(audio, sr, segment_length=7.6):
    segment_samples = int(segment_length * sr)
    segments = []
    for start in range(0, len(audio), segment_samples):
        end = start + segment_samples
        if end <= len(audio):
            segments.append(audio[start:end])
    return segments

# **TEXT******

In [6]:
import numpy as np

# Load GloVe model
def load_glove_model(file_path):
    glove_model = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            split_line = line.strip().split()
            word = split_line[0]
            vector = np.asarray(split_line[1:], dtype='float32')
            glove_model[word] = vector
    return glove_model

# Load the GloVe embeddings (replace 'path_to_glove_file' with the actual file path)
glove_model = load_glove_model('/kaggle/input/glove-model/glove.6B.100d.txt')

In [7]:
import numpy as np

# Simulate GloVe model (dict) with 100D embeddings
glove_model = {"this": np.random.rand(100), "is": np.random.rand(100),
               "a": np.random.rand(100), "test": np.random.rand(100),
               "sentence": np.random.rand(100), "for": np.random.rand(100),
               "GloVe": np.random.rand(100), "embeddings": np.random.rand(100)}

def text_to_embedding(text, max_words=9):
    words = text.split()
    vecs = [glove_model.get(w.lower(), np.zeros(100)) for w in words[:max_words]]
    
    # Pad with zero vectors if there are fewer than `max_words` words
    while len(vecs) < max_words:
        vecs.append(np.zeros(100))
    
    # Stack into (9, 100), then transpose to (100, 9)
    emb = np.stack(vecs, axis=0).T  # (100, 9)
    
    # Pad to (378, 9)
    if emb.shape[0] < 378:
        pad_rows = np.zeros((378 - emb.shape[0], emb.shape[1]))
        emb = np.vstack((emb, pad_rows))  # (378, 9)
    
    return emb

In [8]:
# Example usage
text = clean_text("This is a test sentence for GloVe embeddings")
embedding = text_to_embedding(text)
print(embedding.shape)  # Expected output: (378, 9)

(378, 9)


In [9]:
def build_early_fusion_LSTM(input_shape_audio=(378,60), input_shape_text=(378,9)):
    input_audio = Input(shape=input_shape_audio)
    x_audio = LSTM(60, return_sequences=True)(input_audio)
    x_audio = BatchNormalization()(x_audio)
    x_audio = Dropout(0.2)(x_audio)
    x_audio = Flatten()(x_audio)

    x_text = Input(shape=input_shape_text)
    #x_text = LSTM(60, return_sequences=True)(input_text)
    #x_text = BatchNormalization()(x_text)
    #x_text = Dropout(0.2)(x_text)
    #x_text = Flatten()(x_text)

    merged = Concatenate()([x_audio, x_text])
    dense = Dense(15, activation='tanh')(merged)
    dense = Dense(10, activation='tanh')(dense)
    output = Dense(2, activation='sigmoid')(dense)

    return Model(inputs=[input_audio, input_text], outputs=output)


In [10]:
def build_model_level_fusion_LSTM(input_shape_audio=(378,60), input_shape_text=(378,9)):
    input_audio = Input(shape=input_shape_audio)
    x_audio = LSTM(60, return_sequences=True)(input_audio)
    x_audio = BatchNormalization()(x_audio)
    x_audio = Dropout(0.2)(x_audio)

    x_text = Input(shape=input_shape_text)
    #x_text = LSTM(60, return_sequences=True)(input_text)
    #x_text = LSTM(40, return_sequences=True)(x_text)
    #x_text = LSTM(20)(x_text)

    fusion = Concatenate()([x_audio, x_text])
    fusion = LSTM(60, return_sequences=True)(fusion)
    fusion = BatchNormalization()(fusion)
    fusion = Dropout(0.2)(fusion)
    fusion = Flatten()(fusion)
    fusion = Dropout(0.2)(fusion)
    fusion = Dense(15, activation='tanh')(fusion)
    fusion = Dense(10, activation='tanh')(fusion)
    output = Dense(2, activation='sigmoid')(fusion)

    return Model(inputs=[input_audio, input_text], outputs=output)


In [11]:
model = build_model_level_fusion()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit([X_audio_train, X_text_train], y_train, epochs=30, batch_size=32, validation_split=0.1)


NameError: name 'build_model_level_fusion' is not defined

In [ ]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Concatenate

def build_early_fusion_CNN(input_shape_audio=(378, 60), input_shape_text=(378, 9)):
    # Audio Input
    input_audio = Input(shape=input_shape_audio)
    
    # Audio CNN Layers
    x_audio = Conv2D(128, (3, 3), activation='relu')(input_audio)
    x_audio = MaxPooling2D(pool_size=(2, 2))(x_audio)
    
    x_audio = Conv2D(64, (3, 3), activation='relu')(x_audio)
    x_audio = MaxPooling2D(pool_size=(2, 2))(x_audio)
    x_audio = Dropout(0.02)(x_audio)
    
    x_audio = Flatten()(x_audio)

    # Text Input
    x_text = Input(shape=input_shape_text)
    
    # Text CNN Layers
    #x_text = Conv2D(128, (3, 3), activation='relu')(input_text)
    #x_text = MaxPooling2D(pool_size=(2, 2))(x_text)
    #x_text = Dropout(0.02)(x_text)
    
    #x_text = Conv2D(64, (3, 3), activation='relu')(x_text)
    #x_text = MaxPooling2D(pool_size=(2, 2))(x_text)
    #x_text = Dropout(0.02)(x_text)
    
    x_text = Flatten()(x_text)

    # Early Fusion (Concatenate the audio and text features)
    merged = Concatenate()([x_audio, x_text])
    
    # Fully connected layers after fusion
    dense = Dense(15, activation='tanh')(merged)
    #dense = Dropout(0.02)(dense)  # Dropout to reduce overfitting
    dense = Dense(10, activation='tanh')(dense)
    output = Dense(2, activation='sigmoid')(dense)  # Assuming a binary classification task

    # Define the model
    model = Model(inputs=[input_audio, input_text], outputs=output)
    
    return model

In [ ]:
# Example usage:
input_shape_audio = (378, 60)  # Example input shape for the audio (378x60 spectrogram)
input_shape_text = (378, 9)    # Example input shape for the text (9 words)

model = build_early_fusion_CNN(input_shape_audio, input_shape_text)
model.summary()
